In [1]:
import pandas as pd

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [3]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/egoluback/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
# os.path.getsize('res2.csv')

In [ ]:
train = pd.read_parquet('data_fusion_train.parquet')

In [6]:
train.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
0,7,5,09:50,Флуконазол капс 150мг №1 Вертекс,1.0,7,-1,-1,вертекс
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,None
2,39,4,11:28,"Борщ Станичный с тушенкой, 103 ккал (завод, з/у)",1.0,6,1,-1,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,None


In [7]:
train.shape

(26094077, 9)

In [8]:
train.item_name.value_counts()

Пакет                                         2283982
ЗЕЛПМ-КА32Х62Х17                                59320
Доставка                                        55346
Бананы 1кг Эквадор                              37615
БАНАНЫ ВЕСОВЫЕ                                  22215
                                               ...   
Вафельный сэндвич Ириска вес (ИП Губарева)          1
Троксерутин Врамед гель 2% 40г ( '1', 24)           1
Пирог творожно-абрикосовый 350 г.                   1
Сет №5 Mix                                          1
Усл. парк.  15:56- 11:24                            1
Name: item_name, Length: 3154892, dtype: int64

In [9]:
duplicated = train[train.item_name.duplicated()]

In [10]:
duplicated

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
71,131,1,08:21,Пакет,1.000,3,1,203,None
84,190,1,14:26,Пакет,1.000,1,-1,-1,None
95,227,3,20:17,Пакет,1.000,2,6,-1,None
111,289,4,13:02,Пакет,1.000,2,-1,-1,None
150,356,2,13:07,Пакет,1.000,3,6,203,None
...,...,...,...,...,...,...,...,...,...
45729302,10016159,4,12:39,"Батон ""Молочный"" 0,3",1.000,8,-1,-1,None
45729303,10016159,4,12:39,Виноград кишмиш кг #32<2531>,0.711,11,6,-1,None
45729304,10016159,4,12:39,"Карам.""Ломтик""апел/лимон/грейфр. 1кг Яшкино",0.283,10,-1,-1,None
45729310,10016159,4,12:39,СМЕТАНА 20% Залесский Фермер 200 г. пл.ст.,1.000,7,6,-1,залесский фермер


In [11]:
item_price_duplicated = duplicated.groupby(['item_name', 'item_quantity'])['item_price']

In [12]:
item_price_duplicated.mean()

item_name                                                   item_quantity
                                                            0.017            14.0
                                                            0.040            16.0
                                                            0.044            17.0
                                                            0.050            14.0
                                                            0.052            13.0
                                                                             ... 
№9 ЛК с Грибами в соевом соусе,400гр                        1.000            11.0
№9 СластенаЭ. полосатик пломбир+клубн. б/гл БЗМЖ 60гр 1/30  1.000             7.0
№9 краб                                                     2.000             7.0
┬зПорог Разноуровневый "ЛУКА" Клен беленый                  1.000            11.0
┬зПорог Стыкоперекрыв. "ЛУКА"                               2.000            12.0
Name: item_price, Length

In [50]:
catId_nds_med = train.groupby('category_id').item_nds_rate.median()

In [51]:
catId_nds_med[catId_nds_med == -1]

category_id
45   -1
Name: item_nds_rate, dtype: int32

In [52]:
c_nds_med_max = catId_nds_med.idxmax()

train[train.category_id == c_nds_med_max]

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
28762,64035,3,16:49,Установка стекла,1.0,17,6,13,None
80524,178694,1,14:09,Развал-схождение(шт),1.0,16,6,13,None
111830,244738,6,08:43,РЕМОНТ ШИНЫ УНИВЕРСАЛЬНОЙ ЗАПЛАТОЙ,1.0,11,1,13,None
129745,285440,3,09:26,Установка стекла,1.0,17,6,13,None
139810,307721,1,14:58,Установка стекла,1.0,18,6,13,None
...,...,...,...,...,...,...,...,...,...
45325429,9118837,0,18:13,Регулировка стояночного тормоза,1.0,13,6,13,None
45350209,9172891,1,10:44,Шиномонтаж,1.0,19,-1,13,None
45572395,9664432,5,18:18,Для ремонта камер и шин Прочее,1.0,12,1,13,None
45628457,9788529,3,17:51,Установка стекла,1.0,19,1,13,None


In [40]:
c_nds_med_min = catId_nds_med.idxmin()

train[train.category_id == c_nds_med_min]

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
861,1709,2,17:25,ТЕРАПЕВТИЧЕСКОЕ ЛЕЧЕНИЕ,1.0,18,-1,45,None
7380,15778,1,16:30,VEN Взятие крови из вены,1.0,12,-1,45,None
8678,18877,0,07:27,БИОХИМИЯ (комп.): Aспартатаминотранс(без НДС),1.0,7,-1,45,None
8679,18877,0,07:27,БИОХИМИЯ (комп.): Аланинаминотрансфе(без НДС),1.0,7,-1,45,None
8680,18877,0,07:27,БИОХИМИЯ (комп.): Креатинин(без НДС),1.0,8,-1,45,None
...,...,...,...,...,...,...,...,...,...
45716870,9988360,2,07:36,36 Фосфатаза щелочная,1.0,11,-1,45,None
45716873,9988360,2,07:36,55 Т4 свободный,1.0,12,-1,45,None
45716874,9988360,2,07:36,8 АлАТ,1.0,10,-1,45,None
45716875,9988360,2,07:36,9 АсАТ,1.0,10,-1,45,None


In [ ]:
train = train[train.category_id != -1].drop_duplicates('item_name')

In [ ]:
train.shape

In [ ]:
train['category_id'].value_counts()

In [ ]:
train[train['category_id'] == 84]['item_nds_rate'].value_counts()

In [ ]:
train['item_nds_rate'].value_counts()

In [ ]:
train['brands'].value_counts()

In [ ]:
train.head()

In [ ]:
train.groupby('category_id')['item_name'].value_counts()[84]

In [13]:
train = pd.read_csv('data_fusion_train.csv', index_col= 0)
train = train[train.category_id != -1].drop_duplicates('item_name')

FileNotFoundError: [Errno 2] No such file or directory: 'data_fusion_train.csv'

In [21]:
train = train.sample(1000)

In [22]:
(train != train).sum(axis = 0)

receipt_id             0
receipt_dayofweek      0
receipt_time           0
item_name              0
item_quantity          0
item_price             0
item_nds_rate          0
category_id            0
brands               854
dtype: int64

In [23]:
train.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
1325382,2904783,0,15:10,"Латте ""to go""",1.0,11,5,70,None
602950,1328717,0,15:21,"Харп 0,5л",1.0,14,6,0,None
31246787,8330487,4,12:02,Салат из огурцов с луком зеленым и маслом,1.0,8,-1,71,None
186567,410188,4,11:58,Лосось (семга) малосольная 20,2.0,7,1,82,None
33555549,3384125,2,12:59,"Стрепсилс Интенсив Медово-Лимонные,т",1.0,14,-1,38,None


In [28]:
stop = stopwords.words('russian')

tfidf = TfidfVectorizer(stop_words=stop, max_features=1000)
X_train = tfidf.fit_transform(train.item_name)

In [29]:
X_train

<1000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 2537 stored elements in Compressed Sparse Row format>

In [34]:
train.shape

(1000, 9)

In [35]:
X_train.T.shape

(1000, 1000)

In [36]:
c = X_train.toarray()

In [37]:
np.concatenate([train.T, c.T]).shape

(1009, 1000)

In [38]:

df = pd.DataFrame({'index': c.row, 'node2': c.col, 'edge_weight': c.data})

AttributeError: 'numpy.ndarray' object has no attribute 'row'

In [39]:
df = pd.DataFrame(np.concatenate([train.T, c.T]))

In [63]:
df = df.T

In [64]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,999,1000,1001,1002,1003,1004,1005,1006,1007,1008
0,11,6,20:34,"Молоко 3,2%,шт",2,8,2,78,None,0,...,0,0,0,0,0,0,0,0,0,0
1,39,4,11:28,"Компот из изюма, 114 ккал",1,4,1,71,None,0,...,0,0,0,0,0,0,0,0,0,0
2,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1,4,1,71,None,0,...,0,0,0,0,0,0,0,0,0,0
3,56,5,11:42,Кофе Капучино Большой Эден 18,1,12,1,70,None,0,...,0,0,0,0,0,0,0,0,0,0
4,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1,7,-1,84,None,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
df.to_csv('data_fusion_train_concatAll.csv')

In [67]:
df = df.drop(3, axis = 1)

In [70]:
y = df[7]
df = df.drop([2, 8, 7], axis = 1)

In [6]:
clf = LogisticRegression(max_iter=400)
# cross_val_score(clf, df, y, cv=3, scoring='f1_weighted')

In [71]:
from sklearn.model_selection import train_test_split
X_train1, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)

In [7]:
clf.fit(X_train, train.category_id)

LogisticRegression(max_iter=400)

In [32]:
from sklearn.metrics import f1_score

In [9]:
f1_score(train.category_id, clf.predict(X_train), average = 'weighted')

0.46825497858760295

In [10]:
model = CatBoostClassifier(depth = 4, num_trees = 15)

In [8]:
df = pd.read_csv('data_fusion_train_concatAll.csv', index_col = 0)
y = df['7']
df = df.drop(['2', '8', '7', '3'], axis = 1)

X_train1, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)

In [ ]:

model.fit(X_train, train.category_id)

In [12]:
f1_score(y_test, model.predict(X_test), average = 'weighted')

0.12790761675391094

In [8]:
import pickle
pickle.dump(tfidf, open('tfidf', 'wb'))
pickle.dump(model, open('clf_task1', 'wb'))